# Hälsostudie

Vi vill använda oss av statistik inferens för att denna hälsostudie med information om deltagarnas ålder, kön, vikt, längd, blodtryck, kolesterolnivå, rökvanor och om de har en viss sjukdom för att göra analys och dra statistiksa slutsatser om deras hälsotillstånd.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.stats.power import TTestIndPower

## Förhandsgranska data

Vi vill börja förhandsgranska våra data för att få en första förståelse av hur datasetet ser ut och om det finns eventuella problem.
Genom att använda df.head(), df.describe() och df.info() kan vi se exempel på observationer, få en statistisk sammanfattning samt information om datatyper och antal värden.
Med df.isna().sum() undersöker vi dessutom om det finns saknade värden som kan behöva hanteras innan vidare analys.

**Källa**: *Videolektioner från kursen Pythonprogrammering och statistisk dataanalys*

In [ ]:
# Öppnar och läser filen
df = pd.read_csv("health_study_dataset.csv")

# Förhandsgranskar datan
display(df.head())
display(df.describe())
display(df.info())

df.isna().sum()

## Beskrivande analys

Vi vill undersöka våra data genom en beskrivande analys för att få en överblick över centrala mått och fördelningar.
Vi vill därför räkna ut medelvärde, median, minimum och maximum för variablerna age, weight, height, systolic_bp och cholesterol.
Dessutom vill vi visualisera resultaten genom tre olika grafer som ger en översikt över rökning, blodtryck och vikt hos populationen från datasetet. 

**Källa**: *Videolektioner från kursen Pythonprogrammering och statistisk dataanalys*, *[Geeks for Geeks](https://www.geeksforgeeks.org/python/how-to-make-a-table-in-python/)*

### Översikt - Statistik för hälsodata

In [ ]:
# Tar fram statistik för hälsodata
columns = ["age", "weight", "height", "systolic_bp", "cholesterol"]
data = df[columns]

statistics = pd.DataFrame({
    "Medel": data.mean(),
    "Median": data.median(),
    "Min": data.min(),
    "Max": data.max()
}).round(2)

statistics.style \
    .set_caption("<h2> Översikt - Statistik för hälsodata") \
    .format("{:.2f}")

### Översikt över rökning, blodtryck och vikt

In [ ]:
# Skapar en figure med 1 rad och 3 kolumner (grafer)
# ========================
figs, axes = plt.subplots(1, 3, figsize=(15, 5), sharex=False, sharey=False)

# SUBPLOT 1: Bar chart för rökare
# ========================
ax1 = axes[0]
counts = df["smoker"].value_counts()
bars = ax1.bar(counts.index, counts.values, color=['salmon', 'skyblue'])

for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2, height + 5, str(int(height)),
             ha='center', va='bottom', fontsize=10)

ax1.set_title("Antal rökare/icke-rökare", fontsize=12)
ax1.set_ylabel("Antal personer", fontsize=11)
ax1.grid(True, axis="y", linestyle="--", alpha=0.5)
ax1.tick_params(axis='x')

# SUBPLOT 2: Histogram systoliskt blodtryck
# ========================
ax2 = axes[1]
ax2.hist(df["systolic_bp"], bins=15, color="lightgreen", edgecolor="grey")

mean_bp = df["systolic_bp"].mean()
median_bp = df["systolic_bp"].median()

ax2.axvline(mean_bp, color='salmon', linestyle='dashed', linewidth=1.5, label=f'Medel: {mean_bp:.1f}')
ax2.axvline(median_bp, color='skyblue', linestyle='dashed', linewidth=1.5, label=f'Median: {median_bp:.1f}')

ax2.set_title("Systoliskt blodtryck", fontsize=12)
ax2.set_xlabel("mmHg", fontsize=11)
ax2.set_ylabel("Antal personer", fontsize=11)
ax2.grid(True, axis="y", linestyle="--", alpha=0.5)
ax2.legend()

# SUBPLOT 3: Boxplot över vikt per kön
# ========================
ax3 = axes[2]
ax3.boxplot(
    [df.loc[df['sex'] == 'F', 'weight'], df.loc[df['sex'] == 'M', 'weight']],
    tick_labels=["Kvinnor (F)", "Män (M)"],
    patch_artist=True,
    boxprops=dict(facecolor='skyblue', color='black'),
    medianprops=dict(color='salmon', linewidth=1)
)

ax3.set_title("Vikt per kön", fontsize=12)
ax3.set_xlabel("Kön", fontsize=11)
ax3.set_ylabel("Vikt (kg)", fontsize=11)
ax3.grid(True, axis="y", linestyle="--", alpha=0.5)

plt.suptitle("Översikt över rökning, blodtryck och vikt", fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## Individer med sjukdomen

Vi vill undersöka andelen personer i vårt dataset som har sjukdomen.
Vi vill därför simulera 1000 slumpmässigt valda personer med samma sannolikhet för sjukdom med hjälp av NumPy.
Slutligen vill vi jämföra den simulerade andelen med den verkliga andelen i datasetet för att se hur väl simuleringen stämmer överens med "verkligheten".

**Källa**: *Videolektioner från kursen Pythonprogrammering och statistisk dataanalys*, *[Geeks for Geeks - Pie Chart](https://www.geeksforgeeks.org/data-science/plot-a-pie-chart-in-python-using-matplotlib/)*, *[Geeks for geeks - random.seed()](https://www.geeksforgeeks.org/python/random-seed-in-python/)*

### Andel personer med och utan sjukdomen

In [ ]:
# Skapar ett cirkeldiagram
# ========================
disease_counts = df["disease"].value_counts().sort_index()

plt.figure(figsize=(3, 3))
plt.pie(
    disease_counts.values,
    labels=["Frisk", "Har sjukdom"],
    autopct="%1.1f%%",
    colors=["skyblue", "salmon"],
    startangle=90,
    explode=(0, 0.03)
)
plt.title("Andel personer med och utan sjukdom", fontsize=12, fontweight='bold')
plt.show()

### Sanolikheten att insjukna

Vi vill ta reda på hur väl en slumpmässig simulering av 1000 personer (baserad på den observerade sannolikheten för sjukdom) stämmer överens med den verkliga andelen i datasetet.

In [ ]:
# Simulerar sannolikheten för sjukdomen - jämför med den verkliga andelen i datasetet

# 1. Verklig andel 
# ========================
disease_part = df["disease"].value_counts()
true_proportion = disease_part[1] / disease_part.sum()
print(f"Andelen med sjukdomen: {true_proportion:.1%}")

# 2. Simulera 1000 personer med samma sannolikhet
# ========================
np.random.seed(42)                                  
simulated_disease = np.random.choice(
    [0, 1],                                         
    size=1000,                                      
    p=[1 - true_proportion, true_proportion]        
)

# 3. Beräkna den simulerande andelen
# ========================
simulated_proportion = simulated_disease.mean()     
print(f"Andelen med sjukdomen i simuleringen: {simulated_proportion:.1%}")

# 4. Jämför verklig och simulerad andel
# ========================
print("=====================================")
print(f"Verklig andel:     {true_proportion:.1%}")
print(f"Simulerad andel:   {simulated_proportion:.1%}")
print(f"Skillnad:          {abs(simulated_proportion - true_proportion):.2%}")


## Konfidensintervall

Vi vill vidare undersöka medelvärdet av systoliskt blodtryck i vårt stickprov och hur osäkert detta estimat är.
Vi vill därför beräkna ett 95 % konfidensintervall för det sanna medelvärdet i populationen.
För att göra detta använder vi metoderna normalapproximation och bootstrap och jämför resultaten.
Normalapproximation bygger på teoretiska antaganden om att datan är normalfördelade. Används bäst när stickprovet är stort (n > 40) och datan verkar konsekvent. 
Bootstrap å andra sidan simulerar osäkerheten genom att upprepade gånger dra nya stickprov (med återläggning) ur datasetet, vilket är en metod som fungerar bra även för små eller skeva dataset.

**Källa**: *Videolektioner från kursen Pythonprogrammering och statistisk dataanalys*, *[Geeks for geeks](https://www.geeksforgeeks.org/python/how-to-calculate-confidence-intervals-in-python/)*, *[MIT OpenCourseWare](https://ocw.mit.edu/courses/14-310x-data-analysis-for-social-scientists-spring-2023/resources/14310x-lecture-13_mp4/)*

In [ ]:
# Beräkning av konfidensintervall för medelvärdet av systolic_bp (sbp)

# Normalapproximation
# ========================
pop_sbp = df["systolic_bp"].dropna()
n = len(pop_sbp)
mean_sbp = pop_sbp.mean()
std_sbp = pop_sbp.std(ddof=1)                # stickprovsstandardavvikelse

# Standardfel (SE = s / sqrt(n)) (= osäkerheten i medelvärde)
se_sbp = std_sbp / np.sqrt(n)

# Beräkna 95 % konfidensintervall med normalapproximation
z = 1.96
ci_lower_sbp = mean_sbp - z * se_sbp
ci_upper_sbp = mean_sbp + z * se_sbp

print(f"Punktestimat (medelvärde):    {mean_sbp:.1f} mmHg")
print(f"Standardfel:                  {se_sbp:.3f} mmHg")
print("=====================================")
print(f"95% konfidensintervall med normalapproximation:       [{ci_lower_sbp:.1f}, {ci_upper_sbp:.1f}] mmHg")

# Bootstrap
# ========================
n_boot = 10_000
boot_means = []

# Kör bootstrap-loop
for i in range(n_boot):
    sample_sbp = np.random.choice(pop_sbp, size=n, replace=True)
    boot_means.append(sample_sbp.mean())
# Konverterar till array
boot_means = np.array(boot_means)

# Beräkna 95 % konfidensintervall med bootstrap
ci_lower_boot = np.percentile(boot_means, 2.5)
ci_upper_boot = np.percentile(boot_means, 97.5)

print("=====================================")
print(f"95% konfidensintervall med bootstrap:                 [{ci_lower_boot:.1f}, {ci_upper_boot:.1f}] mmHg")


### Jämförelse
De två olika metoderna, normalapproximation och bootstrap, ger samma resultat. Det innebär att fördelningen av stickprovsmedelvärden sannolikt är ungefär normal och att standardfelet troligen beräknats korrekt.
Med andra ord bekräftar bootstrapen att normalapproximation fungerar att använda för datasetet. 

## Hypotesprövning

Med hjälp av datasetet vill vi undersöka om rökning påverkar det systoliska blodtrycket hos en person.
Vi utgår ifrån vår **nollhypotes (H₀)** att rökare och icke-rökare har samma genomsnittliga systoliska blodtryck.
Där vår **alternativa hypotes (H₁)** är att rökare har högre genomsnittligt systoliskt blodtryck.

**Metod**: 
Welshs T-test har valts som metod för hypotesprövningen eftersom data över blodtrycksvärden är kontinuerligt och vi vill därför utgå ifrån ett medelvärde snarare än en andel, över standard T-test, eftersom Welshs test inte antar att båda grupperna har lika varians och är ett mer robust test att använda i praktiska analyser. Eftersom vi misstänker att rökare har högre blodtryck gör vi både ett tvåsidigt och ett riktat ensidigt test.

- *T-värde*: anger hur många standardfel skillanden mellan gruppernas medelvärde ligger från noll, ett t-värdd när 0 betyder liten skillnad
- *P-värde*: är ett konfidensvärde (0-1) som ger en indikation på hur säkra vi kan vara på ett reslutat över ett annat, desto närmre noll desto säkrare, 0.05 används ofta som gräns för signifikans. 

**Validering**:
Vi vill även undersöka hur säker hypotesttestet är och om testet har en praktisk viktig skillnad genom att göra en power-analys (ensidigt och tvåsidigt). Standardmåttet för "tillräcklig" power för att en studie ska visa praktisk effekt är ≥0.8. 

**Källa**: *Videolektioner från kursen Pythonprogrammering och statistisk dataanalys*, *[MIT OpenCourseWare](https://ocw.mit.edu/courses/14-310x-data-analysis-for-social-scientists-spring-2023/resources/14310x-lecture-13_mp4/)*

In [ ]:
# Welshs t-test
# ========================

# Delar upp populationen i två grupper - smokers/nonsmokers
smokers = df[df["smoker"] == "Yes"]["systolic_bp"].dropna()
nonsmokers = df[df["smoker"] == "No"]["systolic_bp"].dropna()

# Blodtryck medelvärde 
print("=====================================")
print(f"Medelvärde rökare:      {smokers.mean():.2f} mmHg")
print(f"Medelvärde icke-rökare: {nonsmokers.mean():.2f} mmHg")

# Tvåsidigt t-test
t_stat, p_val_two_sided = stats.ttest_ind(smokers, nonsmokers, equal_var=False)

print("=====================================")
print(f"t-värde:                {t_stat:.3f}")
print(f"p-värde (tvåsidigt):    {p_val_two_sided:.3f}")

# Ensidigt t-test
if smokers.mean() > nonsmokers.mean():
    p_val_one_sided = p_val_two_sided / 2
else:
    p_val_one_sided = 1 - p_val_two_sided / 2

print(f"p-värde (ensidigt):     {p_val_one_sided:.3f}")
print("=====================================")

# Beräkna effektstorleken (Cohen's d)
# ========================
std_pooled = np.sqrt(((smokers.std()**2) + (nonsmokers.std()**2)) / 2)      # Uträkning kombinerad standardavvikelse
effect_size = (smokers.mean() - nonsmokers.mean()) / std_pooled             # Cohen’s d = skillnaden mellan gruppernas medelvärden / pooled standard deviation

# Power-analys
# ========================
analysis = TTestIndPower()                                                # Objekt med metoder som räknar ut power, sample size eller effektstorlek

# Tvåsidigt test
power_two_sided = analysis.power(
    effect_size=effect_size,
    nobs1=len(smokers),
    ratio=len(nonsmokers) / len(smokers),
    alpha=0.5,
    alternative="two-sided"
)

# Ensidigt test
power_one_sided = analysis.power(
    effect_size=effect_size,
    nobs1=len(smokers),
    ratio=len(smokers) / len(smokers),
    alpha=0.5,
    alternative="larger"
)

print(f"Power (tvåsidigt):      {power_two_sided:.3f}")
print(f"Power (ensidigt):       {power_one_sided:.3f}")
print("=====================================")

### Tolkning av reslatet

Ett **t-värde** på 0.450 är väldigt nära noll, vilket betyder att skillnaden i blodtryck mellan rökare och icke-rökare är mycket liten i förhållande till variationen i data.

Det **tvåsidiga p-värdet** (0.653) är mycket högt, vilket betyder att vi har ingen statistisk evidens för att rökare och icke-rökare skiljer sig åt i blodtryck. Det **ensidiga p-testet** (0.326) testar den riktade hypotesen, eftersom resultatet fortfarande är mycket högre än gränsen för signifikans (0.05), så går det inte att förkasta nollhypotesen.

**Power-analysen** visar för det *tvåsidiga testet* att undersökningen har en medelstor effekt (0.543), däremot ligger det långt från det önskvärda måttet på ≥0.8. Det *ensidiga testet* visar en högre power, men även med detta testet är chansen att missa en verklig skillnad fortfarande relativt hög.

**Slutsats:** P-värdena visar att den observerade skillnaden i blodtryck mellan rökare och icke-rökare lika gärna kan bero på slumpen. Eftersom p-värdena är mycket större än 0.05 finns ingen statistiskt säkerställd skillnad i blodtryck mellan rökare och icke-rökare. Power-värdena visar på att resultatet av studien är osäkert och kan vara svårt att reproducera. Eftersom p-värdena också är höga (inte signfikanta) så blir en låg power särskild anmärkningsvärt eftersom studien antagligen inte har tillräckligt med data för att upptäcka en verklig skillnad. 